<a href="https://colab.research.google.com/github/aminul01-g/people-flow-tracker-yolov8/blob/main/ObjectTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q ultralytics opencv-python tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00


In [2]:
#Import libraries
from ultralytics import YOLO
import cv2
import numpy as np
import os
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Load model
model = YOLO('yolov8x.pt')
print("model loaded")

model loaded


In [4]:
# Download video
!wget -O people-walking.mp4 https://media.roboflow.com/supervision/video-examples/people-walking.mp4
video_path = 'people-walking.mp4'

--2025-08-06 01:46:41--  https://media.roboflow.com/supervision/video-examples/people-walking.mp4
Resolving media.roboflow.com (media.roboflow.com)... 34.110.133.209
Connecting to media.roboflow.com (media.roboflow.com)|34.110.133.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7606633 (7.3M) [video/mp4]
Saving to: ‘people-walking.mp4’

people-walking.mp4  100%[===================>]   7.25M  --.-KB/s    in 0.05s   

2025-08-06 01:46:41 (135 MB/s) - ‘people-walking.mp4’ saved [7606633/7606633]



In [5]:
# Initialize capture
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [6]:
# Output video setup (use AVI/XVID for VLC compatibility)
output_path = 'annotated_video_tracked.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, video_fps, (width, height))

In [7]:
# Define lines
top_line = np.array([[15, 367], [1919, 367]])
bottom_line = np.array([[15, 623], [1910, 605]])
top_y = int(np.mean(top_line[:, 1]))
bottom_y = int(np.mean(bottom_line[:, 1]))


In [8]:
id_positions = {}
id_states = {}
in_count = 0
out_count = 0

In [9]:
# Heatmap canvas
heatmap_canvas = np.zeros((height, width), dtype=np.float32)
fade_alpha = 0.95

In [10]:
# Frame loop
for i in tqdm(range(frame_count), desc="Processing video"):
    ret, frame = cap.read()
    if not ret:
        break

    # Perform tracking
    results = model.track(frame, classes=[0], persist=True, verbose=False)
    boxes = results[0].boxes.xyxy.tolist()
    track_ids = results[0].boxes.id.int().tolist() if results[0].boxes.id is not None else []

    current_ids = set()
    cv2.line(frame, tuple(top_line[0]), tuple(top_line[1]), (255, 0, 0), 2)
    cv2.line(frame, tuple(bottom_line[0]), tuple(bottom_line[1]), (0, 0, 255), 2)

    print(f"[Frame {i}] Detections: {len(boxes)}, IDs: {track_ids}")

    for box, track_id in zip(boxes, track_ids):
        x1, y1, x2, y2 = map(int, box)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        current_ids.add(track_id)

        if track_id in id_positions:
            prev_y = id_positions[track_id]
            if prev_y < top_y and cy >= top_y and id_states.get(track_id) != "in":
                in_count += 1
                id_states[track_id] = "in"
            elif prev_y > bottom_y and cy <= bottom_y and id_states.get(track_id) != "out":
                out_count += 1
                id_states[track_id] = "out"

        id_positions[track_id] = cy

        color = (255, 255, 255)
        if id_states.get(track_id) == "in":
            color = (0, 255, 0)
        elif id_states.get(track_id) == "out":
            color = (0, 0, 255)

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.circle(frame, (cx, cy), 5, color, -1)

        # Heatmap accumulation
        cv2.circle(heatmap_canvas, (cx, cy), 8, 1, -1)

    # Remove lost IDs
    lost_ids = set(id_positions.keys()) - current_ids
    for lost_id in lost_ids:
        id_positions.pop(lost_id, None)
        id_states.pop(lost_id, None)

    # Heatmap overlay
    heatmap_canvas *= fade_alpha
    temp_heatmap = np.clip(heatmap_canvas, 0, 255)
    temp_heatmap = cv2.GaussianBlur(temp_heatmap, (31, 31), 0)
    heatmap_color = cv2.applyColorMap(temp_heatmap.astype(np.uint8), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(frame, 0.7, heatmap_color, 0.5, 0)

    # Add counters
    cv2.putText(overlay, f"IN: {in_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(overlay, f"OUT: {out_count}", (20, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(overlay, f"Total: {len(current_ids)}", (20, 140), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    out.write(overlay)

cap.release()
out.release()

Processing video:   0%|          | 0/341 [00:00<?, ?it/s]

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



Processing video:   0%|          | 1/341 [00:09<55:38,  9.82s/it]

[Frame 0] Detections: 35, IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


Processing video:   1%|          | 2/341 [00:10<23:33,  4.17s/it]

...

Processing video:  99%|█████████▉| 339/341 [00:58<00:00,  7.45it/s]

[Frame 337] Detections: 34, IDs: [7, 14, 17, 24, 29, 71, 74, 75, 97, 98, 132, 149, 152, 157, 161, 184, 186, 188, 192, 198, 99, 183, 216, 218, 219, 223, 245, 250, 263, 266, 269, 271, 272, 275]
[Frame 338] Detections: 35, IDs: [7, 14, 17, 24, 29, 71, 74, 75, 97, 98, 132, 149, 152, 157, 161, 184, 186, 188, 192, 198, 99, 183, 216, 218, 219, 223, 245, 250, 263, 266, 269, 271, 272, 275, 241]


Processing video: 100%|██████████| 341/341 [00:58<00:00,  7.45it/s]

[Frame 339] Detections: 35, IDs: [7, 14, 17, 24, 29, 71, 74, 75, 97, 98, 132, 149, 152, 157, 161, 184, 186, 188, 192, 198, 99, 183, 216, 218, 219, 223, 245, 250, 263, 266, 269, 271, 272, 275, 241]
[Frame 340] Detections: 34, IDs: [7, 14, 17, 24, 29, 74, 75, 97, 98, 132, 149, 152, 157, 161, 184, 186, 188, 192, 198, 99, 183, 216, 218, 219, 223, 245, 250, 263, 266, 269, 271, 272, 275, 241]


Processing video: 100%|██████████| 341/341 [00:58<00:00,  5.79it/s]


In [11]:
#Config
video_path = '/content/people-walking.mp4'
output_heatmap_video = 'heatmap_output.mp4'
output_image_path = 'final_heatmap.jpg'
model_path = 'yolov8n.pt'
conf_threshold = 0.3

In [12]:
# Load model
model = YOLO(model_path)

In [13]:
# Video setup
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    raise ValueError("Could not read from video.")

H, W = frame.shape[:2]
heatmap_canvas = np.zeros((H, W), dtype=np.float32)

In [14]:
# Output video setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_heatmap_video, fourcc, 30.0, (W, H))

In [15]:
# prcess video frame
while ret:

    results = model(frame, verbose=False)[0]

    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        if cls_id == 0 and conf > conf_threshold:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            cv2.circle(heatmap_canvas, (cx, cy), radius=20, color=3, thickness=-1)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)

    #Create real-time heatmap overlay
    temp_map = np.clip(heatmap_canvas * 15, 0, 255).astype(np.uint8)
    blurred = cv2.GaussianBlur(temp_map, (31, 31), 0)
    colored_map = cv2.applyColorMap(blurred, cv2.COLORMAP_JET)

    overlay = cv2.addWeighted(frame, 0.6, colored_map, 0.4, 0)

    out.write(overlay)

    ret, frame = cap.read()

cap.release()
out.release()

In [16]:
# Normalize and amplify final heatmap
amplified_heatmap = np.clip(heatmap_canvas * 15, 0, 255)
normalized_heatmap = cv2.normalize(amplified_heatmap, None, 0, 255, cv2.NORM_MINMAX)
blurred_heatmap = cv2.GaussianBlur(normalized_heatmap.astype(np.uint8), (31, 31), 0)
colored_heatmap = cv2.applyColorMap(blurred_heatmap, cv2.COLORMAP_JET)

In [17]:
# Save final heatmap image
cv2.imwrite(output_image_path, colored_heatmap)
print(f"Final heatmap image saved: {output_image_path}")
print(f"Heatmap video saved: {output_heatmap_video}")

Final heatmap image saved: final_heatmap.jpg
Heatmap video saved: heatmap_output.mp4
